In [ ]:
import numpy as np

In [ ]:
#FUNCION IM2COL EN PYTHON (FUNCIONES PARA PODER HACER LA CONVOLUCION MUCHO MÁS RAPIDO)
#FUNCION PARA UNA SOLA IMAGEN DE 2 DIMENSIONES
def vec2D(A):
  blocksize = A.shape
  stepsize = 1
  # Parameters
  M,N = A.shape
  col_extent = N - blocksize[1] + 1
  row_extent = M - blocksize[0] + 1
    
    # Get Starting block indices
  start_idx = (np.arange(blocksize[0])[:,None]*N + np.arange(blocksize[1])).T
    
    
    # Get offsetted indices across the height and width of input array
  offset_idx = (np.arange(row_extent)[:,None]*N + np.arange(col_extent)).T
    
    # Get all actual indices & index into input array for final output
  return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

#FALTA DEFINIR FUNCION PARA VECTORIZAR ARRAYS 3D

def im2Dcol(A, blocksize, stride, stepsize=1):
  # Parameters
  M,N = A.shape
  col_extent = int((N - blocksize[1])/stride + 1)
  row_extent = int((M - blocksize[0])/stride + 1)
    
    # Get Starting block indices
  start_idx = (np.arange(blocksize[0])[:,None]*N + np.arange(blocksize[1])).T
    
    # Get offsetted indices across the height and width of input array
  offset_idx = ((np.arange(row_extent)[:,None]*N + np.arange(col_extent))*stride).T
    
    # Get all actual indices & index into input array for final output
  return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

#FUNCION DE EXPANSION PARA UN CONJUNTO DE IMAGENES DE C CANALES
def imNDcol(A, blocksize, stride, maxpool = False, stepsize=1):
  m, n_C, n_H, n_W = A.shape
  #blocksize = [3,2,2]
  col_extent = int((n_W - blocksize[2])/stride + 1)
  row_extent = int((n_H - blocksize[1])/stride + 1)
      
  # Get Starting block indices
  start_idx = (np.arange(blocksize[1])[:,None]*n_W + np.arange(blocksize[2])).T
  start_idx = np.broadcast_to(start_idx, (1,n_C,blocksize[1], blocksize[2]))
  idx_s = np.arange(0,n_W*n_W*n_C,n_W*n_W)[:, None]
  idx_s = idx_s[None, :]
  idx_s = idx_s[:,:,np.newaxis]
  start_idx = start_idx + idx_s 

  # Get offsetted indices across the height and width of input array
  offset_idx = ((np.arange(row_extent)[:,None]*n_W + np.arange(col_extent))*stride).T
  h_o, w_o = offset_idx.shape
  offset_idx = np.broadcast_to(offset_idx, (m, 1, h_o, w_o))
  idx_o = np.arange(0,n_W*n_H*n_C*m, n_W*n_H*n_C)[:, None]
  idx_o = idx_o[:,:,np.newaxis,np.newaxis]
  offset_idx = offset_idx + idx_o
  
  # Get all actual indices & index into input array for final output
  indices = start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize]

  if maxpool:
    return np.take(A, indices), indices
  else:
    return np.take(A, indices)

In [ ]:
# FUNCIONES PARA LA CONVOLUCIÓN PERO SIN BUCLES FOR(Se trata de usar en lo mínimo posible el ciclo for)
# imgIn => (#ejemplos)x(#canales)x(ancho)x(altura)
# Filters => (#filtros)x(#canales)x(ancho)x(altura)
def conv4D(imgIn, filters, padding, stride):
  #ESTA FUNCION BOTA AL CONVOLUCIÓN PERO NO CON LA FORMA QUE DEBERÍA BOTAR (FALTA MODIFICAR)
  #PERO EL CÁLCULO ES CORRECTO
  m, n_C, n_H, n_W = imgIn.shape
  n_f, f_C, f_H, f_W = filters.shape
  n = int((n_H - f_H)/stride + 1)

  B = imNDcol(imgIn, [f_C, f_H, f_W], stride = stride)
  F = imNDcol(filters, [f_C, f_H, f_W], stride = stride)

  #Convolución
  conv = B.T @ F

  convsplit = np.split(conv, m, axis = 0)
  for i in range(m):
    convsplit[i] = convsplit[i].T.reshape((n_f,n,n), order = "F")
  CONV = np.stack(convsplit) 
  return CONV

#(EL MAXPOOLING SE HACE A CADA CANAL DE LA IMAGEN DE ENTRADA)
def maxpooling(X, filsize, padding, stride):
  m, n_C, n_H, n_W = X.shape
  blocksize = [n_C, filsize, filsize]
  indX = np.arange(m*n_C*n_H*n_W).reshape((m, n_C, n_H, n_W))
  BMAX = np.zeros(X.shape)

  #Se obtiene la matriz de expansión y los indices que ayudarán a la derivada
  B, indices= imNDcol(X, blocksize, stride, maxpool = True)
  PHIindX = imNDcol(indX, blocksize, 1)

  #Ciclo for para hallar los máximos
  #Entre todas las filas de B están distribuidas los n_C canales
  #Bpool contiene los maximos 
  Bpool = np.zeros((n_C, B.shape[1]))
  Bmax = np.zeros(indices.shape)
  k = 0
  step = filsize**2

  #maxpooling
  for i in range(n_C):
    Bpool[i] = np.amax(B[i+k:i+step+k], axis = 0, keepdims = True)
    ind_max = np.argmax(B[i+k:i+step+k], axis = 0)[None, :]
    np.put_along_axis(Bmax[i+k:i+step+k], ind_max, 1, axis = 0)
    k = k + step - 1
  
  # devuelve la matriz con la misma forma pero solo con máximos (esto ayuda al backpropagation)
  indMAX = np.extract(PHIindX*Bmax > 0, PHIindX*Bmax).astype(int)
  for i in range(len(indMAX)):
    ind = np.unravel_index(indMAX[i], X.shape)
    BMAX[ind] = X[ind]

  return Bpool, BMAX

#listo!!!
def Mindicator(X, OX, filsize):
  m, n_C, n_H, n_W = X.shape
  p, q = OX.shape

  M = np.zeros((p*q, m*n_C*n_H*n_W))

  Xind = np.arange(m*n_C*n_H*n_W).reshape((m,n_C,n_H,n_W))
  PHIXind = imNDcol(Xind, [n_C, filsize, filsize], 1)

  PHIXindflat = vec2D(PHIXind) 
  Xindflat = imNDcol(Xind, [n_C, n_H, n_W], 1)
  Xindflat = vec2D(Xindflat).T

  for pq in range(p*q):
    M[pq] = np.where(Xindflat == PHIXindflat[pq], 1, 0)

  return M

def flatten(imgIn):
  (m,n_C,n_H,n_W) = imgIn.shape
  x = imgIn.reshape((n_C*n_H*n_W,m))
  return x

#FUNCIONES DE ACTIVACIÓN Y DE PÉRDIDA
relu = (lambda x : np.maximum(x, 0),
        lambda x : (x > 0) * 1)
f_loss_logistic = (lambda y,a: -np.mean(y*np.log(a[-1])+(1-y)*np.log(1-a[-1])) , 
          lambda y,a: -(y/a - (1-y)/(1-a)))
f_sigmoid = (lambda z: 1/(1+np.exp(-z)), 
             lambda z: z*(1-z))

#FUNCIONES PARA LA RED NEURONAL TOTALMENTE CONECTADA
def init_parameters(L, struct):
  teta = []
  b = np.random.rand(L-1,1)*2-1

  for i in range(L-1):
    teta_aux = np.random.rand(struct[i+1],struct[i])*2-1
    teta.append(teta_aux)

  return teta, b

def forward(x_train, teta, b, f_act, L):
  out = []
  out.append(x_train)
  for i in range(L-1):
    if i == L-2:
      y_pred, y_prob = softmax(out, teta, b)
      out.append(y_prob)
    else:
      z = teta[i] @ out[-1] + b[i]
      a = f_act[0](z)
      out.append(a)

  return out, y_pred


def backward(y_train, out, f_loss, f_act, L, teta, b, lr):
  deltas = []
  for i in range(L-2, -1, -1):
    if i == L-2:
      #delta = out[-1]-y_train
      delta = f_loss[1](y_train, out[-1]) * f_act[1](out[-1])
      deltas.insert(0,delta)
    else:
      delta = (_teta.T @ delta)*f_act[1](out[i+1])
      #delta = (teta[i+1].T @ delta)*f_act[1](out[i+1])
      deltas.insert(0,delta)
    
    _teta = teta[i]
    # ACTUALIZACION DE PARÁMETROS 
    teta[i] = teta[i] - lr*(deltas[0] @ out[i].T)
    b[i] = b[i] - lr*np.mean(deltas[0])
  
  return deltas

# NEURONA SOFTMAX
def softmax(outputs, teta, b):
  z = teta[-1] @ outputs[-1] + b[-1]
  t = np.exp(z)
  suma = np.sum(t, axis = 0, keepdims = True)
  for i in range(t.shape[1]):
    t[:,i] = t[:,i]/suma[:,i]
  y_prob = t.astype(float)
  indice = np.argmax(t, axis = 0)
  for i in range(t.shape[1]):
    t[:,i] = 0
    t[:,i][indice[i]] = 1
  return t, y_prob

def rotate3D(filters):
  for i in range(filters.shape[0]):
    filters[i] = filters[i][::-1,::-1]
  return filters

def rotate4D(filters):
  for i in range(filters.shape[0]):
    filters[i] = filters[i][::-1,::-1,::-1]
  return filters

def padding3D(a,f,c):
  b = np.pad(a, ((0,0), (f,f), (c,c)), 'constant')
  return b

def padding4D(a,f,c):
  b = np.pad(a, ((0,0), (0,0), (f,f), (c,c)), 'constant')
  return b